### Sentence Unique Code

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk import word_tokenize
import re

In [ ]:
df_sent = pd.read_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Pyspark/Result/Sentence/Spark Turkish Sentence Merge/Clean_Spark_Sentence_Merge_Result.csv")
df_sent = df_sent.head(500)
df_sent

In [ ]:
# # 1-Way
# count_index = 0
# code_list = []
# for i in df_sent.sentence:
#     g = len(re.findall("\w+", i))
#     l = len(i)
#     f = df_sent.iloc[count_index].frequency
#     unique_code = f"G{g}-L{l}-F{f}-{count_index}"
#     code_list.append(unique_code)
#     count_index += 1
# df_sent["code"] = pd.DataFrame(code_list)
# df_sent

In [ ]:
# 2-Way
# df_empty = pd.DataFrame({'col_name' : []})
df_sent["code"] = pd.Series(dtype="string")
count_index = 0
for i in df_sent.sentence:
    g = len(re.findall("\w+", i))
    l = len(i)
    f = df_sent.iloc[count_index].frequency
    unique_code = f"G{g}-L{l}-F{f}-{count_index}"
    df_sent.at[count_index, "code"] = unique_code
    count_index += 1
df_sent

In [ ]:
df_sent.iloc[33]

#### Multi Processing

In [1]:
import pandas as pd
import numpy as np
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re

In [2]:
help(multiprocessing)

Help on package multiprocessing:

NAME
    multiprocessing

MODULE REFERENCE
    https://docs.python.org/3.9/library/multiprocessing
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

DESCRIPTION
    # Package analogous to 'threading.py' but using processes
    #
    # multiprocessing/__init__.py
    #
    # This package is intended to duplicate the functionality (and much of
    # the API) of threading.py but uses processes instead of threads.  A
    # subpackage 'multiprocessing.dummy' has the same API but is a simple
    # wrapper for 'threading'.
    #
    # Copyright (c) 2006-2008, R Oudkerk
    # Licensed to PSF under a Contributor Agreement.
    #

PACKAGE CONTENTS
    connection
    context
    dummy (packag

In [3]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [4]:
df_sent = pd.read_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Pyspark/Result/Sentence/Spark Turkish Sentence Merge/Clean_Spark_Sentence_Merge_Result_Clean_Digit_One_Char.csv")
df_sent = df_sent.head(500)
df_sent

,sentence,frequency
0,evet,1948596
1,hayır,1250401
2,tamam,882921
3,ne,753105
4,teşekkürler,343527
...,...,...
495,ne hakkında,10109
496,hassiktir,10101
497,sen de,10089
498,beğendin mi,10056


In [5]:
index_sent = zip(df_sent.sentence.index, df_sent.sentence)
zip_list = list(index_sent)
#zip_list

In [7]:
# index, sentence = zip(*index_sent)

In [16]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
code_list = manager.list()
sent_list = [] = manager.list()
freq_list = [] = manager.list()

def sentence_code(index_num, sent):
    g = len(re.findall("\w+", sent))
    l = len(sent)
    f = df_sent.iloc[index_num].frequency
    unique_code = f"G{g}-L{l}-F{f}-{index_num}"
    sent_list.append(sent)
    freq_list.append(f)
    code_list.append(unique_code)
    
    

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.starmap(sentence_code, zip_list) # zip_list fonksiyon için çoklu eleman gerekli olduğu için kullanıldı.
# Not: starmap çoklu iterable değerlere ihtiyaç duyan fonksitonlar için kullanılıyor ve bu değerler zip veya itertools şelinde olmalı. O yüzden burada pool.map yerine pool.starmap kullanıldı. 

In [33]:
list_sent = list(sent_list)
#list_sent

In [34]:
list_code = list(code_list)
#list_code

In [ ]:
list_freq = list(freq_list)
#list_code

In [35]:
df_code = pd.DataFrame({"sentence":list_sent,
                        "frequency":list_freq,
                        "code":list_code})

In [37]:
df_code.sort_values(by="frequency", ascending=False)

,sentence,code
0,haklısın,G1-L8-F60602-72
1,ben de,G2-L6-F59972-73
2,aman tanrım,G2-L11-F189521-16
3,nasılsın,G1-L8-F66410-56
4,neden,G1-L4-F1948596-0
...,...,...
495,i̇lginç,G2-L7-F10430-477
496,hatırladın mı,G2-L13-F10114-494
497,bu iyi,G2-L6-F10399-478
498,her zaman,G2-L9-F10399-479


In [41]:
pd.merge(df_sent,df_code, how="inner", on="sentence")

,sentence,frequency,code
0,evet,1948596,G2-L11-F59827-74
1,hayır,1250401,G2-L5-F66394-57
2,tamam,882921,G1-L6-F95836-43
3,ne,753105,G1-L2-F753105-3
4,teşekkürler,343527,G1-L6-F75303-51
...,...,...,...
495,ne hakkında,10109,G2-L11-F10109-495
496,hassiktir,10101,G2-L10-F10250-488
497,sen de,10089,G2-L8-F11695-421
498,beğendin mi,10056,G2-L7-F10660-467
